Import Torch Packages

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as T
import torch.optim as optim

#### Import Gym Packages

In [2]:
from ale_py import ALEInterface
ale = ALEInterface()

A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]


In [3]:
from ale_py.roms import Breakout
ale.loadROM(Breakout)

Game console created:
  ROM file:  /home/beegass/.virtualenvs/dl_1/lib/python3.8/site-packages/AutoROM/roms/breakout.bin
  Cart Name: Breakout - Breakaway IV (1978) (Atari)
  Cart MD5:  f34f08e5eb96e500e851a80be3277a56
  Display Format:  AUTO-DETECT ==> NTSC
  ROM Size:        2048
  Bankswitch Type: AUTO-DETECT ==> 2K

Running ROM file...
Random seed is 1647576012


In [4]:
import gym

#### All Other Packages

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
import random
import copy
import typing
from typing import Callable
import PIL 
from PIL import Image
from abc import ABC, abstractmethod
from loguru import logger

/home/beegass/.virtualenvs/dl_1/lib/python3.8/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


##### The Meat And Potatoes 

In [6]:
class Gym_Env():
    def __init__(self, env_name, max_steps=1000, max_episodes=10000):
        self.le_env = gym.make(env_name)
        self.max_steps = max_steps
        self.max_episodes = max_episodes
        

In [7]:
class Replay_Buffer():
    def __init__(self, capacity, mini_batch_size=128):
        self.rb = []
        self.capacity = capacity
        self.mini_batch_size = mini_batch_size
        self.current_batch = None

    def sample_rb(self):
        return random.sample(self.rb, batch_size=self.mini_batch_size)
    
    def add_to_rb(self, new_transition):
        if len(self.rb) >= self.capacity:
            del self.rb[0] 
        self.rb.append(new_transition)

In [8]:
# ripped from https://nn.labml.ai/rl/dqn/index.html
class Deep_Q_Network(nn.Module):
    def __init__(self):
        super(Deep_Q_Network, self).__init__()
        
        self.network = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        self.lin = nn.Linear(in_features=7 * 7 * 64, out_features=512)
        self.activation = nn.ReLU()
        
        self.state_value = nn.Sequential(
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=1)
            )
        
        self.action_value = nn.Sequential(
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=4),
        )
    
    def forward(self, x):
        x = self.network(x)
        x = x.reshape((-1, 7 * 7 * 64))
        x = self.activation(self.lin(x))
        action_value = self.action_value(x)
        state_value = self.state_value(x)
        action_score_centered = action_value - action_value.mean(dim=-1, keepdim=True)
        q = state_value + action_score_centered
        return q  
        
    

In [9]:
class Agent(nn.Module):
    def __init__(self, pred_model):
        super(Agent, self).__init__()
        self.agent = pred_model
        self.target = None
        
    def agent_policy(self, state, grad=False):
        q_val = -1
        if not grad:
            with torch.no_grad():
                q_val = self.agent(state)
        else:
            q_val = self.target(state)
        return torch.argmax(q_val), q_val
    
    def copy_pred_to_target(self):
        self.target = copy.deepcopy(self.agent)
    

##### Epsilon

In [10]:
def epsilon_greedy(environment, the_agent, state, epsilon=0.1):
    prob = random.random()
    action = environment.le_env.action_space.sample() # pick action from action space
    if prob < 1 - epsilon:
        action, _ = the_agent.agent_policy(state) # retrieve best action, based off its action-value action
    return action

In [11]:
def epsilon_decay(environment, episode_num, p_init=0.7, p_end=0.1):
    episode_num += 1
    current_episode_rate = (environment.max_episodes - episode_num) / environment.max_episodes
    epsilon_decay_rate = max(current_episode_rate, 0)
    return ((p_init - p_end) * (epsilon_decay_rate) + p_end)

##### Preprocess

In [12]:
def preprocess(state):
    logger.info(f"----Before Processing----")
    logger.info(f"state shape: {state.shape}")
    logger.info(f"state type: {type(state)}")
    frame = torch.from_numpy(state)
    reshaped_frame = frame.reshape(3, 210, 160)
    gray_frame = T.Grayscale()(reshaped_frame)
    small_gray_frame = T.Resize((84, 84))(gray_frame)
    logger.info(f"----After Processing----")
    logger.info(f"state shape: {small_gray_frame.shape}")
    logger.info(f"state type: {type(small_gray_frame)}")
    return small_gray_frame
    

##### Global Variables

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # check if GPU is available
MAX_EPISODES = 10000
MAX_STEPS = 10000
REPLAY_BUFFER_SIZE = 5000
MINI_BATCH_SIZE = 36
EPSILON = 0.1

##### Buidling Optimizers and Schedulers

In [14]:
def build_optimizer(model, optimizer_name='adam', learning_rate=0.01, weight_decay=0.01, momentum=0.9):
    try:
        optimizer = None
        if optimizer_name == "sgd":
            optimizer = optim.SGD(model.parameters(), 
                                  lr=learning_rate, 
                                  momentum=momentum)
            
        elif optimizer_name == "adam":
            optimizer = optim.Adam(model.parameters(), 
                                   lr=learning_rate, 
                                   weight_decay=weight_decay)
               
        return optimizer
    except:
        print("Error: Invalid optimizer specified.")
        sys.exit(1)

In [15]:
def build_scheduler(optimizer, sched_name='reduce_lr', patience=5, verbose=True):
    try: 
        sched = None
        if sched_name == "reduce_lr":
            sched = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                         mode='min', 
                                                         patience=patience, 
                                                         verbose=verbose)
        elif sched_name == 'TODO':
            pass
            #TODO: add other scheduler
            
        return sched
    except:
        logger.error("Error: Invalid scheduler specified.")
        sys.exit(1)

##### Training Loop

In [16]:
def train(replay_buffer, the_agent, optimizer, scheduler):
    mini_batch = replay_buffer.sample_rb()
    
    #1. copy weights from pred to target
    the_agent.copy_pred_to_target()
    
    #2. init y_j
    y_j = 0 
    
    #3. retrieve (s, a, r, s') from mini_batch
    for transition in mini_batch:
        (state, _, next_state, reward, done) = transition
        state = state.to(device)
        _, pred_highest_q_val = the_agent.agent_policy(state, grad=True)
        y_j = torch.FloatTensor([reward])
        if not done:
            _, target_highest_q_val = the_agent.agent_policy(next_state, grad=False)
            y_j += GAMMA * target_highest_q_val
        loss = nn.MSELoss(pred_highest_q_val, y_j.detach())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 
    the_agent.target = None

In [17]:
def run_episode(environment, the_agent, state, epsilon, optimizer, scheduler):
    replay_buffer = Replay_Buffer(capacity=REPLAY_BUFFER_SIZE, mini_batch_size=MINI_BATCH_SIZE)
    
    step_count = 0
    while True:
        
        logger.info(f"state before NN: {type(state)}")
        logger.info(f"state before NN: {state.shape}")
        
        #0. either explore or exploit
        action = epsilon_greedy(environment=environment,
                                the_agent=the_agent,
                                state=state,
                                epsilon=epsilon)
        
        #1. pass action to environment
        (next_state, reward, done, _) = environment.le_env.step(action)
        
        #2. TODO: get s' back from environment and preprocess (s' -> preprocessed_s')
        preprocessed_next_state = preprocess(next_state)
        
        #3. add transition (s, a, s', r) to replay buffer
        replay_buffer.add_to_rb((state, action, preprocessed_next_state, reward, done))
        state = preprocessed_next_state
        
        #4. if replay buffer is full, sample mini batch and update model
        if len(replay_buffer.rb) >= replay_buffer.capacity:
            train(replay_buffer, the_agent, optimizer, scheduler)
        
        #5. check max number of time steps has been reached or if game is complete
        if step_count >= environment.max_steps or done:
            break
        
        state = next_state
        
        step_count += 1
        
        
        

In [18]:
def run(epsilon=EPSILON):
    # initialize gym environment
    environment = Gym_Env(env_name='ALE/Breakout-v5', max_steps=MAX_STEPS, max_episodes=MAX_EPISODES)
    
    # initialize prediction network
    pred_net = Deep_Q_Network(environment.le_env.action_space.n).to(device)
    
    # initialize agent that contains both prediction network and target network
    the_agent = Agent(pred_model=pred_net)
    
    # define loss function
    loss_fn = nn.MSELoss()
    
    # define optimizer
    optimizer = build_optimizer(model=the_agent.agent, 
                                optimizer_name='adam', 
                                learning_rate=0.01, 
                                weight_decay=0.01, 
                                momentum=0.9)
    
    # define scheduler
    scheduler = build_scheduler(optimizer, 
                                sched_name='reduce_lr', 
                                patience=5, 
                                verbose=True)
    
    global EPSILON
    epsilon = EPSILON
    for e in range(environment.max_episodes):
        
        # 0. get initial state, s_{0}, and preprocess it (s_{0} -> preprocessed_s)
        state = environment.le_env.reset()
        
        # 0.1: TODO: preprocess state
        preprocessed_state = preprocess(state)
        
        # 1. iterate over steps in episode
        run_episode(environment=environment, 
                    the_agent=the_agent,
                    state=preprocessed_state, 
                    epsilon=epsilon, 
                    optimizer=optimizer, 
                    scheduler=scheduler)
        
        # 2. decay epsilon
        epsilon = epsilon_decay(environment=environment, episode_num=e)

In [19]:
run()

A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]
/home/beegass/.virtualenvs/dl_1/lib/python3.8/site-packages/gym/utils/seeding.py:138: DeprecationWarning: WARN: Function `hash_seed(seed, max_bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
/home/beegass/.virtualenvs/dl_1/lib/python3.8/site-packages/gym/utils/seeding.py:175: DeprecationWarning: WARN: Function `_bigint_from_bytes(bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
2022-03-18 00:00:16.793 | INFO     | __main__:preprocess:2 - ----Before Processing----
2022-03-18 00:00:16.794 | INFO     | __main__:preprocess:3 - state shape: (210, 160, 3)
2022-03-18 00:00:16.795 | INFO     | __main__:preprocess:4 - state type: <class 'numpy.ndarray'>
2022-03-18 00:00:16.796 | INFO     | __main__:preprocess:9 - ----After Processing----
2022-03-18 00:00:16.797 | INFO     | __main__:preprocess:10 - state shape: torch.Size([1, 84, 84])
2022-03-18 00:0

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 4, 8, 8], but got 3-dimensional input of size [1, 84, 84] instead